### UC OSS Bedrock Agent Demo notebook 
### Shorter version to test existing agents and functions/tools.

### UC OSS Bedrock Agent Demo notebook 
### Shorter version to test existing agents and functions/tools.

1. **Imports and Configuration:**
    - Imports necessary modules and classes from `unitycatalog` and `pprint`.
    - Defines constants for the Unity Catalog server, catalog, and schema.

2. **Client Initialization:**
    - Configures the Unity Catalog API client with the server address.
    - Initializes the `UnitycatalogFunctionClient` with the API client.

3. **Toolkit Initialization:**
    - Defines a list of function names to be used.
    - Initializes the `UCFunctionToolkit` with the function names and client.

4. **Print Toolkit:**
    - Prints the initialized `UCFunctionToolkit` object.

5. **List Functions:**
    - Lists functions available in the specified catalog and schema.
    - Pretty prints the list of functions.

6. **Bedrock Agent Configuration:**
    - Configures the Bedrock agent with agent ID and alias.
    - Creates a session with the Bedrock agent and generates a unique session ID.

7. **Invoke Agent:**
    - Invokes the Bedrock agent with a specific input text to get weather information.
    - Enables trace and uses the generated session ID.

8. **Display Response:**
    - Displays the response body from the agent invocation, which contains the weather information in Celsius and Fahrenheit.

In [1]:
from unitycatalog.ai.core.client import UnitycatalogFunctionClient
from unitycatalog.client import ApiClient, Configuration
from unitycatalog.ai.bedrock.toolkit import UCFunctionToolkit
import pprint

In [2]:
UNITY_CATALOG_SERVER = "localhost:8080"
CATALOG = "AICatalog1"
SCHEMA = "AISchema1"

In [3]:
config = Configuration()
config.host = f"http://{UNITY_CATALOG_SERVER}/api/2.1/unity-catalog"

# The base ApiClient is async
api_client = ApiClient(configuration=config)

uc_client = UnitycatalogFunctionClient(api_client=api_client)

In [4]:
function_names = [f"{CATALOG}.{SCHEMA}.get_weather_in_celsius", f"{CATALOG}.{SCHEMA}.get_weather_in_fh"]
uc_f_toolkit = UCFunctionToolkit(function_names=function_names, client=uc_client)

In [5]:
print(uc_f_toolkit)

function_names=['AICatalog1.AISchema1.get_weather_in_celsius', 'AICatalog1.AISchema1.get_weather_in_fh'] tools_dict={'AICatalog1.AISchema1.get_weather_in_celsius': BedrockTool(name='AICatalog1__AISchema1__get_weather_in_celsius', description='Test function for AWS Bedrock integration.', parameters={'type': 'object', 'properties': {'location_id': {'default': None, 'description': 'The name to be included in the greeting message.', 'title': 'Location Id', 'type': 'string'}, 'fetch_date': {'default': None, 'description': 'The date with the location', 'title': 'Fetch Date', 'type': 'string'}}, 'required': []}, requireConfirmation='ENABLED'), 'AICatalog1.AISchema1.get_weather_in_fh': BedrockTool(name='AICatalog1__AISchema1__get_weather_in_fh', description='Test function for AWS Bedrock integration.', parameters={'type': 'object', 'properties': {'location_id': {'default': None, 'description': 'The name to be included in the greeting message.', 'title': 'Location Id', 'type': 'string'}, 'fetch

In [6]:
uc_functions = uc_client.list_functions(
    catalog=CATALOG,
    schema=SCHEMA,
    max_results=10
)

pprint.pp(uc_functions)

[FunctionInfo(name='get_weather_in_celsius', catalog_name='AICatalog1', schema_name='AISchema1', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='location_id', type_text='STRING', type_json='{"name": "location_id", "type": "string", "nullable": false, "metadata": {"comment": "The name to be included in the greeting message."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The name to be included in the greeting message.'), FunctionParameterInfo(name='fetch_date', type_text='STRING', type_json='{"name": "fetch_date", "type": "string", "nullable": false, "metadata": {"comment": "The date with the location"}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=1, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The date wi

### Toolkit usage with single action group and multiple functions/tools

In [7]:
import boto3, pprint, json, time, uuid

# Bedrock agent configuration
agent_id = "HXPGL1POGD"
agent_alias_id_c_fh = "4PF6BYFZBW"

session_fh = uc_f_toolkit.create_session(agent_id=agent_id,
                                agent_alias_id=agent_alias_id_c_fh,
                                catalog_name=CATALOG,
                                schema_name=SCHEMA)
# Generate unique session ID
session_id_cfh = str(uuid.uuid1())

Initialized BedrockSession with agent_id: HXPGL1POGD, agent_alias_id: 4PF6BYFZBW, catalog_name: AICatalog1 schema_name: AISchema1, 


In [8]:
final_response = session_fh.invoke_agent(
                input_text="What is the weather in Centigrade and Fahrenheit for location 1234 and date of 2024-11-19?",
                enable_trace=True,
                session_id=session_id_cfh,
                uc_client = uc_f_toolkit.client
                )

****************************
Invoking the agent with params:
Timestamp: 2025-02-25 23:07:33
{'agentId': 'HXPGL1POGD', 'agentAliasId': '4PF6BYFZBW', 'inputText': 'What is the weather in Centigrade and Fahrenheit for location 1234 and date of 2024-11-19?', 'enableTrace': True, 'sessionId': '4ba0f728-f3cd-11ef-91b8-dae3f3b23e8c'}
****************************
Response from invoke agent: {'ResponseMetadata': {'RequestId': '36c4e5e1-3a19-4022-acaa-7abc45edd7d7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 25 Feb 2025 23:07:33 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': '36c4e5e1-3a19-4022-acaa-7abc45edd7d7', 'x-amz-bedrock-agent-session-id': '4ba0f728-f3cd-11ef-91b8-dae3f3b23e8c', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': '4ba0f728-f3cd-11ef-91b8-dae3f3b23e8c', 'completion': <botocore.eventstream.EventStr

In [9]:
final_response.response_body

'The weather for location 1234 on 2024-11-19 is 23°C (Celsius) and 72°F (Fahrenheit).'